## Import Packages

In [2]:
using Oceananigans #use v.1.10
using Oceananigans.Units
using Oceananigans.OutputReaders: FieldTimeSeries
using Oceananigans.BoundaryConditions
using Oceananigans.TurbulenceClosures
using NCDatasets

┌ Warning: You are using Julia v1.11 or later!"
│ Oceananigans is currently tested on Julia v1.10."
│ If you find issues with Julia v1.11 or later,"
│ please report at https://github.com/CliMA/Oceananigans.jl/issues/new
└ @ Oceananigans C:\Users\Tom Cummings\.julia\packages\Oceananigans\3ZIHr\src\Oceananigans.jl:124


## Define Model Parameters

In [3]:
# Model parameters
Nx = 300
Nz = 80
f = 1e-4               # Coriolis frequency [s⁻¹]
L_front = 10kilometers  # Initial front width [m]
aspect_ratio = 100      # L/H
δ = 0                   # Strain ratio (α/f)
Ro = 1              # Rossby number (defines M^2)
F = Inf              # Froude number (N² = M²/(F²H))
Re_h = 500    # horizontal reynolds number
Re_v = +Inf              #vertical reynolds number
n = 2              #diffusivity number                 

sponge_width = 8kilometers
damping_rate = f

# Derived parameters
H_front = L_front/aspect_ratio
α = f*δ
M² = (Ro^2*f^2*L_front)/H_front
N² = (M²*L_front)/(F^2*H_front)
Bu = Ro/F
Δb = M²*L_front
κh = (sqrt(Δb*H_front)*L_front^(n-1))/Re_h
κv = κh*(Re_h/Re_v)*(H_front/L_front)^n




filename = "δ="*string(δ)*"_Ro="*string(Ro)*"_F="*string(F)*"_Re_h="*string(Re_h)
filename = "B00_sin_IC10"

println("Filename: ", filename)
println("\nDerived parameters:")
println("H = ", H_front/1000, " km")
println("α = ", α, " s⁻¹")
println("M² = ", M², " s⁻²")
println("N² = ", N², " s⁻²")
println("Bu = ", Bu)
println("κh = ", κh)
println("κv = ", κv)

Filename: B00_sin_IC10

Derived parameters:
H = 0.1 km
α = 0.0 s⁻¹
M² = 1.0e-6 s⁻²
N² = 0.0 s⁻²
Bu = 0.0
κh = 20.0
κv = 0.0


## Define Model Domain

In [4]:

Lx = 64*L_front #40kilometers
Lz = H_front

grid = RectilinearGrid(size = (Nx, Nz), #in ST15 they use Nx = 200*L_front, Nz = 100*H_front
                       x = (-L_front/2, L_front/2),
                       z = (-Lz, 0),
                       topology = (Periodic, Flat, Bounded))

300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CPU with 3×0×3 halo
├── Periodic x ∈ [-5000.0, 5000.0) regularly spaced with Δx=33.3333
├── Flat y                         
└── Bounded  z ∈ [-100.0, 0.0]     regularly spaced with Δz=1.25

## Forcing Terms

In [5]:
# advective forcing term 
u_background = XFaceField(grid)
u_background .= - α * xnodes(grid, Face(), Center(), Center())
background_flow = AdvectiveForcing(u = u_background)

# no addtional u forcing

# v forcing
v_forcing_func(x, z, t, v, α) = - 2*α*v
v_forcing = Forcing(v_forcing_func, parameters = α, field_dependencies = :v )

# w forcing
w_forcing_func(x, z, t, w, α) = - α*w
w_forcing = Forcing(w_forcing_func, parameters = α, field_dependencies = :w )

# b forcing
b_forcing_func(x, z, t, α, b ) = - α*b
b_forcing = Forcing(b_forcing_func, parameters = α, field_dependencies= :b )



ContinuousForcing{Float64}
├── func: b_forcing_func (generic function with 1 method)
├── parameters: 0.0
└── field dependencies: (:b,)

## Sponge Layer

In [6]:
Δb = L_front * M²

target_buoyancy_left(x,z,t) = z*N²
target_buoyancy_right(x,z,t) = z*N² + Δb
    




left_mask_3D   = GaussianMask{:x}(center=-grid.Lx/2, width = sponge_width)
left_mask(x,z) = left_mask_3D(x,0,z)
uvw_sponge_left = Relaxation(rate=damping_rate, mask=left_mask)
b_sponge_left = Relaxation(rate=damping_rate, mask=left_mask, target = target_buoyancy_left) 

right_mask_3D  = GaussianMask{:x}(center=grid.Lx/2,width = sponge_width)
right_mask(x,z) = right_mask_3D(x,0,z)
uvw_sponge_right = Relaxation(rate=damping_rate, mask=right_mask)
b_sponge_right = Relaxation(rate=damping_rate, mask=right_mask, target = target_buoyancy_right ) 



Relaxation{Float64, typeof(right_mask), typeof(target_buoyancy_right)}
├── rate: 0.0001
├── mask: right_mask (generic function with 1 method)
└── target: target_buoyancy_right (generic function with 1 method)

## Boundary Conditions

In [7]:

using Oceananigans.BoundaryConditions

# Free-slip for u and v (∂u/∂z = ∂v/∂z = 0)
free_slip = FieldBoundaryConditions(
    top = GradientBoundaryCondition(0.0),
    bottom = GradientBoundaryCondition(0.0)
)

# No vertical flow (w = 0 at top/bottom)
no_penetration = FieldBoundaryConditions(
    top = ValueBoundaryCondition(0.0),
    bottom = ValueBoundaryCondition(0.0)
)

velocity_bcs = (
    u = free_slip,
    v = free_slip,
    w = no_penetration
)

(u = Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: GradientBoundaryCondition: 0.0
├── top: GradientBoundaryCondition: 0.0
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing), v = Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: GradientBoundaryCondition: 0.0
├── top: GradientBoundaryCondition: 0.0
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing), w = O

## Buoyancy BC's

In [8]:

∂b∂z_top = N²
∂b∂z_bottom = N²

buoyancy_bcs = FieldBoundaryConditions(
    top = GradientBoundaryCondition(∂b∂z_top),
    bottom = GradientBoundaryCondition(∂b∂z_bottom)
)
    

Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: GradientBoundaryCondition: 0.0
├── top: GradientBoundaryCondition: 0.0
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)

## Closure

In [9]:
horizontal_closure = HorizontalScalarDiffusivity(ν=κh, κ=κh )
vertical_closure = VerticalScalarDiffusivity(ν=κv , κ=κv )
closure = (horizontal_closure, vertical_closure)

(HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=20.0, κ=20.0), VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0, κ=0.0))

## Defining the Model

In [10]:
#with sponge

#=
model = NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = :b,
                advection = WENO(),
                forcing = (; u = (background_flow, uvw_sponge_left, uvw_sponge_right),
                             v = (background_flow , v_forcing, uvw_sponge_left, uvw_sponge_right) , 
                             w = (background_flow , w_forcing, uvw_sponge_left, uvw_sponge_right),
                             b = (background_flow, b_forcing, b_sponge_left, b_sponge_right)),
                boundary_conditions = (; b=buoyancy_bcs, velocity_bcs),
                closure = closure

                )
=#

#without sponge 

model = NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = :b,
                advection = WENO(),
                forcing = (; u = (background_flow),
                             v = (background_flow , v_forcing) , 
                             w = (background_flow , w_forcing),
                             b = (background_flow, b_forcing))
                #boundary_conditions = (; b=buoyancy_bcs, velocity_bcs)
                )





NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO{3, Float64, Float32}(order=5)
├── tracers: b
├── closure: Nothing
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

## Initial Conditions

In [11]:
#inital setup

Δb = L_front * M²  # buoyancy jump across front  

function bᵢ(x, z)
    if x < -pi*L_front/2
        return N² * z
    elseif x > pi*L_front/2
        return N² * z + Δb
    else
        return N² * z + Δb*(sin(x/L_front) + 1)/2
    end
end


set!(model, b= bᵢ, u = 0, v = 0, w = 0)  # Start from rest


## Define Simulation

In [ ]:
simulation = Simulation(model, Δt=20minutes, stop_time=10days)

In [ ]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [ ]:
using Printf

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, max(u): (%6.3e, %6.3e, %6.3e) m/s, next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, v), maximum(abs, w), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(100))

## Diagnostic/Output

In [ ]:
# Output setup
g = 9.81
u, v, w = model.velocities
ζ = ∂z(u) - ∂x(w)  # Vorticity in x-z plane
b = model.tracers.b

# Compute Ri
Ri = N² / (∂z(u)^2 + ∂z(v)^2) 
M_squared = ∂x(b)

#X = x + v/f
dbdX = ∂x(b)*(1/(1 + (1/f) * ∂x(v)))




#=
#For Julia animation
simulation.output_writers[:fields] = JLD2Writer(
    model, (; b, ζ , u, v, w),
    filename=filename * ".jld2",
    schedule=TimeInterval(0.5day),
    overwrite_existing=true
    )
=#
#=

#For Lagrangian filtering
simulation.output_writers[:fields] = JLD2Writer(
    model, (; b, u, v, w, Ri, M_squared, dbdX), filename=filename * ".jld2", schedule=TimeInterval(10minutes), overwrite_existing=true)
=#


#for python visualisation
simulation.output_writers[:fields] = NetCDFWriter(
    model, (; b, u, v, w, Ri, M_squared, dbdX), filename=filename * ".nc", schedule=TimeInterval(10minutes), overwrite_existing=true)

## Run Simulation

In [ ]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)